In [2]:
import networkx as nx
import random
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from gensim.models import Word2Vec
import pandas as pd

Objective: Determine the function where optimal D = f(m, n)

In [ ]:
results = []

for c in range(1, 10): # 9
    for connect_num in range(1, 5): # 4
        for npc in range(8, 40, 4): # 8
            for epc in range(16, 5136, 64): # 80
                for window in range(1, 10, 2): # 7
                    n = c * npc
                    m = c * epc
                    D = 2 * m / (n * (n-1))
                    test_size = int(m * 0.2)
                    if m >= n and D < 1 and test_size > 5:
                        G=nx.Graph()
                        train_G=nx.Graph()

                        nodes = [x for x in range(n)]
                        G.add_nodes_from(nodes)
                        train_G.add_nodes_from(nodes)

                        for i in range(c):
                            edges_in_clique = 0
                            while edges_in_clique <= epc:
                                a = random.randint(npc*i, npc*(i+1))
                                b = random.randint(npc*i, npc*(i+1))
                                if a != b:
                                    G.add_edge(a, b)
                                    train_G.add_edge(a, b)
                                    edges_in_clique += 1

                        # Create an edge for any singleton nodes
                        limit = 0
                        for node in nodes:
                            while len(list(G.neighbors(node))) < connect_num:
                                b = random.randint(0, n-1)
                                limit += 1
                                if b != node:
                                    G.add_edge(node, b)
                                    train_G.add_edge(node, b)
                                if limit > n*20:
                                    break

                        test_edges = []
                        labels = []
                        limit = 0
                        while len(test_edges) < test_size:
                            h_u, h_v = random.choice(list(train_G.edges))
                            u_neighbors = len(list(train_G.neighbors(h_u)))
                            v_neighbors = len(list(train_G.neighbors(h_v)))
                            if u_neighbors > connect_num and v_neighbors > connect_num:
                                train_G.remove_edge(h_u, h_v)
                                test_edges.append((h_u, h_v))
                                labels.append(1)
                                limit -= 1
                            limit +=1 
                            if limit > n*20:
                                break

                        limit = 0
                        while len(test_edges) < test_size*2:
                            a = random.randint(0, n-1)
                            b = random.randint(0, n-1)
                            if G.has_edge(a, b) == False and a != b:
                                test_edges.append((a , b))
                                labels.append(0)
                                limit -= 1
                            limit +=1
                            if limit > n*20:
                                break

                        walk_length = 80
                        walks_per_node = 10

                        walks = []
                        for node in nodes:
                            for walk in range(walks_per_node):
                                nodes_in_walk = [str(node)]
                                cur_node = node
                                for step in range(walk_length - 1):
                                    neigh = list(train_G.neighbors(cur_node))
                                    next_step = random.choice(neigh)
                                    cur_node = next_step
                                    nodes_in_walk.append(str(next_step))
                                walks.append(nodes_in_walk)

                        for dim_order in range(1, 9):
                            model = Word2Vec(size=2**dim_order, window=window, workers=8, 
                                           ns_exponent =-0.5,
                                           sg=1, hs=0, negative=5)
                            model.build_vocab(walks)
                            losses = []
                            aucs = []
                            for i in range(100):
                                model.train(walks, total_examples=len(walks),
                                        epochs=1, compute_loss=True)
                                losses.append(model.get_latest_training_loss())
                                preds = []
                                for t in test_edges:
                                    pred = model.wv.similarity(str(t[0]), str(t[1]))
                                    preds.append(pred)
                                auc = roc_auc_score(labels, preds)
                                aucs.append(auc)
                            results.append([c, D, npc, epc, connect_num, window, 2**dim_order, np.mean(losses[-5:]), min(losses), np.mean(aucs[-5:]), max(aucs)])

                        if len(results) % 250 == 0:
                            print(np.array(walks).shape) 
                            print("Nodes:", n)
                            print("Edges:", m)
                            print("Graph Density:", D)
                            print("Connectivity:", connect_num)
                            print("Cliques:", c)
                            print("Window:", window)
                            print("Test size:", test_size)    
                            print(2**dim_order, np.mean(losses[-5:]), min(losses), np.mean(aucs[-5:]), max(aucs))
                            columns = ["cliques", "density", "nodes per clique", "edges per clique", 
                                       "connectivity between cliques", "w2v window", "w2v dimensions", 
                                       "average loss", "min loss", "average auc", "max auc"]
                            results_df = pd.DataFrame(results, columns = columns)
                            results_df.to_csv('HyperParameter_Results.csv')

(360, 80)
Nodes: 36
Edges: 400
Graph Density: 0.6349206349206349
Connectivity: 1
Cliques: 1
Window: 9
Test size: 80
256 49206.0046875 41452.65234375 0.5761875 0.5910937500000001
(360, 80)
Nodes: 36
Edges: 208
Graph Density: 0.33015873015873015
Connectivity: 2
Cliques: 1
Window: 9
Test size: 41
256 48255.25546875 40692.01953125 0.4926829268292682 0.5907198096371208
(320, 80)
Nodes: 32
Edges: 464
Graph Density: 0.9354838709677419
Connectivity: 3
Cliques: 1
Window: 9
Test size: 92
256 41759.65078125 24519.12109375 0.4811436672967864 0.5847117202268431
(320, 80)
Nodes: 32
Edges: 272
Graph Density: 0.5483870967741935
Connectivity: 4
Cliques: 1
Window: 9
Test size: 54
256 45581.83671875 24865.583984375 0.5270233196159122 0.6646090534979424
(480, 80)
Nodes: 48
Edges: 160
Graph Density: 0.14184397163120568
Connectivity: 1
Cliques: 2
Window: 9
Test size: 32
256 50223.84140625 42006.0 0.88359375 0.9189453125
(640, 80)
Nodes: 64
Edges: 928
Graph Density: 0.4603174603174603
Connectivity: 1
Cliques

In [10]:
columns = ["cliques", "density", "nodes per clique", "edges per clique", 
           "connectivity between cliques", "w2v window", "w2v dimensions", 
           "average loss", "min loss", "average auc", "max auc"]
results = pd.DataFrame(results, columns = columns).sort_values(by='average auc', ascending=False)

In [16]:
results.to_csv('Results 12518.csv')



1.   What is the minimum number of nodes & edges that makes embeddings reasonable?
2.   What is the optimal size of embedding space as a function of nodes and edges



